### 패션 MNIST

In [2]:
# 데이터 불러오기
from torchvision import datasets
from torchvision.transforms import ToTensor

In [3]:
# 학습 데이터 불러오기
train_data = datasets.FashionMNIST(
                root = "./Data", # 데이터가 저장될 경로 
                train = True, # True이면 학습데이터, False이면 테스트 데이터 
                download = True, # True이면 데이터가 없을 경우 자동 다운로드 
                transform = ToTensor() # 이미지를 텐서로 변환 
)

100%|██████████| 26.4M/26.4M [00:00<00:00, 114MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 4.12MB/s]
100%|██████████| 4.42M/4.42M [00:00<00:00, 58.3MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 11.1MB/s]


In [4]:
# 테스트 데이터 불러오기
test_data = datasets.FashionMNIST(
                root = "./Data", # 데이터가 저장될 경로 
                train = False, # True이면 학습데이터, False이면 테스트 데이터 
                download = True, # True이면 데이터가 없을 경우 자동 다운로드 
                transform = ToTensor() # 이미지를 텐서로 변환 
)

In [5]:
# Data와 Target의 변수 재정의
train_input = train_data.data
train_target = train_data.targets

test_input = test_data.data
test_target = test_data.targets

In [6]:
train_scaled = train_input / 255.0 
train_scaled = train_scaled.reshape(-1, 28*28) # [60000, 28, 28]

test_scaled = test_input / 255.0 
test_scaled = test_scaled.reshape(-1, 28*28) # [60000, 28, 28]

In [7]:
# Dimension 확인

print(train_scaled.shape)
print(test_scaled.shape)

torch.Size([60000, 784])
torch.Size([10000, 784])


In [8]:
# Train data를 훈련데이터와 검증 데이터로 나누기

from sklearn.model_selection import train_test_split

train_scaled, val_scaled, train_target, val_target = train_test_split(
                                                        train_scaled, 
                                                        train_target,
                                                        test_size = 0.2,
                                                        random_state = 42
)

In [9]:
# 분리후의 Dimension 확인
print(train_scaled.shape)
print(train_target.shape)
print(val_scaled.shape)
print(val_target.shape)

torch.Size([48000, 784])
torch.Size([48000])
torch.Size([12000, 784])
torch.Size([12000])


----
### 심층 신경망 만들기

In [10]:
# Module 불러오기
import torch
import torch.nn as nn # Neural Network
import torch.optim as optim # Optimizer(ex: SGD, Adam, ...)
from torch.utils.data import TensorDataset, DataLoader

In [11]:
# Dataset 및 DataLoader 생성

batch_size = 32 # mini batch
train_dataset = TensorDataset(train_scaled, train_target)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

val_dataset = TensorDataset(val_scaled, val_target)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

---
### 모델 정의


In [12]:
# 입력층 -> 은닉층(100)(활성화함수: ReLU) -> 출력층(Softmax)
class FashionMNISTModel1(nn.Module):
    def __init__(self):
        super(FashionMNISTModel1, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(28*28, 100)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(100, 10)
        self.softmax = nn.Softmax(dim=1)
    def forward(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.softmax(x)
        return x     

----

In [28]:
# 입력층 -> 은닉층(100)(활성화함수: ReLU) -> Dropout층 -> 출력층(Softmax)
class FashionMNISTModel2(nn.Module):
    def __init__(self):
        super(FashionMNISTModel2, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(28*28, 100)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(100, 10)
        self.softmax = nn.Softmax(dim=1)
    def forward(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.dropout1(x)
        x = self.fc2(x)
        x = self.softmax(x)
        return x     

---

In [36]:
# 모델 정의 : 입력층 -> 은닉층(100,활성화함수)-> Dropout층(0.2) -> 은닉층(50,활성화함수) -> 출력층
class FashionMNISTModel3(nn.Module):
    def __init__(self):
        super(FashionMNISTModel3, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(28*28, 100)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(0.2)
        self.fc2 = nn.Linear(100, 50)
        self.relu2 = nn.ReLU()        
        self.fc3 = nn.Linear(50, 10)
        self.softmax = nn.Softmax(dim=1)
    def forward(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.dropout1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        x = self.softmax(x)
        return x     

---

In [ ]:
# 모델 정의 : 
# 입력층 -> 은닉층(100,활성화함수)-> Dropout층(0.2) -> 은닉층(50,활성화함수) 
# -> Dropout층(0.2) -> 출력층

In [37]:
# 모델, 손실함수, 옵티마이저 

model1 = FashionMNISTModel1()
criterion1 = nn.CrossEntropyLoss() # 손실 함수 
optimizer1 = optim.Adam(model1.parameters())

model2 = FashionMNISTModel2()
criterion2 = nn.CrossEntropyLoss() # 손실 함수 
optimizer2 = optim.Adam(model2.parameters())

model3 = FashionMNISTModel3()
criterion3 = nn.CrossEntropyLoss() # 손실 함수 
optimizer3 = optim.Adam(model3.parameters())

In [14]:
# 학습 함수 정의
def train(model, train_loader, criterion, optimizer, device):
    model.train() # 뉴런들이 훈련모드로 동작하도록 설정 
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad() # 이전 반복에서 계산된 그래디언트를 초기화 
        outputs = model(inputs) # 내부적으로 model.forward(inputs) 호출하여 예측값 얻음 
        loss = criterion(outputs, targets) # 모델의 예측값과 실제 타겟 간의 손실을 계산 
        loss.backward() # 손실에 대한 그래디언트를 계산하고 역전파 
        optimizer.step() # 계산된 그래디언트를 사용하고 모델의 파라미터를 업데이트 
    return loss.item() # 현재 배치의 손실값을 반환 

In [15]:
# 평가함수 
def evaluate(model, val_loader, criterion, device):
    model.eval() # 평가모드로 동작하도록 설정 
    total_loss = 0 # 전체 손실 합계 
    correct = 0 # 정확하게 예측한 샘플 수 
    total = 0 # 전체 샘플 수
    with torch.no_grad() : # 평가 중에는 그래디언트 계산이 필요 없으므로 메모리 사용량 및 연산 속도 향상
        for inputs, targets in val_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs) # 예측값 발생 
            loss = criterion(outputs, targets) # 손실계산
            total_loss += loss.item() # 전체 손실에 더하기 
            _, predicted = outputs.max(1) # 전체 샘플에 대해 가장 높은 확률을 가진 인덱스 
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item() # 예측값과 실제값이 일치하는 경우 True를 발생 
        return total_loss/len(val_loader), correct/total # 평균손실과 정확도 

In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [38]:
# model to device
model1.to(device)
model2.to(device)
model3.to(device)

FashionMNISTModel3(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=784, out_features=100, bias=True)
  (relu1): ReLU()
  (dropout1): Dropout(p=0.2, inplace=False)
  (fc2): Linear(in_features=100, out_features=50, bias=True)
  (relu2): ReLU()
  (fc3): Linear(in_features=50, out_features=10, bias=True)
  (softmax): Softmax(dim=1)
)

In [18]:
# 훈련 Loop - Model 1

num_epochs = 50

for epoch in range(num_epochs):
    train_loss = train(model1, train_loader, criterion1, optimizer1, device)
    print(f"Epoch[{epoch+1}/{num_epochs}], Loss :{train_loss:.4f}")

Epoch[1/50], Loss :1.6163
Epoch[2/50], Loss :1.6332
Epoch[3/50], Loss :1.5029
Epoch[4/50], Loss :1.4833
Epoch[5/50], Loss :1.6145
Epoch[6/50], Loss :1.6339
Epoch[7/50], Loss :1.5373
Epoch[8/50], Loss :1.5915
Epoch[9/50], Loss :1.6233
Epoch[10/50], Loss :1.6795
Epoch[11/50], Loss :1.6313
Epoch[12/50], Loss :1.5834
Epoch[13/50], Loss :1.5786
Epoch[14/50], Loss :1.6490
Epoch[15/50], Loss :1.6557
Epoch[16/50], Loss :1.7723
Epoch[17/50], Loss :1.5531
Epoch[18/50], Loss :1.5619
Epoch[19/50], Loss :1.5473
Epoch[20/50], Loss :1.5905
Epoch[21/50], Loss :1.5408
Epoch[22/50], Loss :1.5857
Epoch[23/50], Loss :1.5545
Epoch[24/50], Loss :1.5291
Epoch[25/50], Loss :1.4647
Epoch[26/50], Loss :1.5250
Epoch[27/50], Loss :1.5655
Epoch[28/50], Loss :1.5169
Epoch[29/50], Loss :1.5348
Epoch[30/50], Loss :1.4612
Epoch[31/50], Loss :1.5547
Epoch[32/50], Loss :1.5592
Epoch[33/50], Loss :1.4890
Epoch[34/50], Loss :1.4613
Epoch[35/50], Loss :1.5875
Epoch[36/50], Loss :1.5241
Epoch[37/50], Loss :1.5255
Epoch[38/5

In [31]:
# 훈련 Loop - Model 2

num_epochs = 50

for epoch in range(num_epochs):
    train_loss = train(model2, train_loader, criterion2, optimizer2, device)
    print(f"Epoch[{epoch+1}/{num_epochs}], Loss :{train_loss:.4f}")

Epoch[1/50], Loss :1.5870
Epoch[2/50], Loss :1.6013
Epoch[3/50], Loss :1.7587
Epoch[4/50], Loss :1.6318
Epoch[5/50], Loss :1.5433
Epoch[6/50], Loss :1.5258
Epoch[7/50], Loss :1.6095
Epoch[8/50], Loss :1.6125
Epoch[9/50], Loss :1.5583
Epoch[10/50], Loss :1.6240
Epoch[11/50], Loss :1.6346
Epoch[12/50], Loss :1.5675
Epoch[13/50], Loss :1.6359
Epoch[14/50], Loss :1.6176
Epoch[15/50], Loss :1.6051
Epoch[16/50], Loss :1.7822
Epoch[17/50], Loss :1.5641
Epoch[18/50], Loss :1.5549
Epoch[19/50], Loss :1.5752
Epoch[20/50], Loss :1.5832
Epoch[21/50], Loss :1.6124
Epoch[22/50], Loss :1.5907
Epoch[23/50], Loss :1.6841
Epoch[24/50], Loss :1.5409
Epoch[25/50], Loss :1.6833
Epoch[26/50], Loss :1.6254
Epoch[27/50], Loss :1.5256
Epoch[28/50], Loss :1.6497
Epoch[29/50], Loss :1.6159
Epoch[30/50], Loss :1.5797
Epoch[31/50], Loss :1.5844
Epoch[32/50], Loss :1.5427
Epoch[33/50], Loss :1.5829
Epoch[34/50], Loss :1.6573
Epoch[35/50], Loss :1.5696
Epoch[36/50], Loss :1.6045
Epoch[37/50], Loss :1.6490
Epoch[38/5

---

In [39]:
# 훈련 Loop - Model 3

num_epochs = 50

for epoch in range(num_epochs):
    train_loss = train(model3, train_loader, criterion3, optimizer3, device)
    print(f"Epoch[{epoch+1}/{num_epochs}], Loss :{train_loss:.4f}")

Epoch[1/50], Loss :1.6470
Epoch[2/50], Loss :1.5813
Epoch[3/50], Loss :1.5913
Epoch[4/50], Loss :1.5805
Epoch[5/50], Loss :1.6317
Epoch[6/50], Loss :1.6320
Epoch[7/50], Loss :1.6576
Epoch[8/50], Loss :1.5622
Epoch[9/50], Loss :1.4927
Epoch[10/50], Loss :1.5866
Epoch[11/50], Loss :1.5538
Epoch[12/50], Loss :1.5549
Epoch[13/50], Loss :1.5571
Epoch[14/50], Loss :1.6145
Epoch[15/50], Loss :1.6123
Epoch[16/50], Loss :1.5250
Epoch[17/50], Loss :1.6158
Epoch[18/50], Loss :1.5804
Epoch[19/50], Loss :1.6040
Epoch[20/50], Loss :1.5503
Epoch[21/50], Loss :1.6017
Epoch[22/50], Loss :1.5865
Epoch[23/50], Loss :1.5564
Epoch[24/50], Loss :1.5205
Epoch[25/50], Loss :1.5566
Epoch[26/50], Loss :1.5235
Epoch[27/50], Loss :1.5237
Epoch[28/50], Loss :1.5628
Epoch[29/50], Loss :1.5862
Epoch[30/50], Loss :1.5237
Epoch[31/50], Loss :1.6767
Epoch[32/50], Loss :1.5656
Epoch[33/50], Loss :1.6450
Epoch[34/50], Loss :1.6496
Epoch[35/50], Loss :1.5237
Epoch[36/50], Loss :1.4671
Epoch[37/50], Loss :1.5555
Epoch[38/5

----
### Model 1
: 입력층 -> 은닉층(활성화함수) -> 출력층

In [19]:
# 훈련 평가
train_loss, train_accuracy = evaluate(model1, train_loader, criterion1, device)
print(f"Loss : {train_loss:.4f}, Accuracy : {train_accuracy:.4f}")

Loss : 1.5449, Accuracy : 0.9164


In [20]:
# 검증 평가
val_loss, val_accuracy = evaluate(model1, val_loader, criterion1, device)
print(f"Loss : {val_loss:.4f}, Accuracy : {val_accuracy:.4f}")

Loss : 1.5797, Accuracy : 0.8818


In [21]:
# 일반화 평가
test_dataset = TensorDataset(test_scaled, test_target)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

test_loss, test_accuracy = evaluate(model1, test_loader, criterion1, device)
print(f"Loss : {test_loss:.4f}, Accuracy : {test_accuracy:.4f}")

Loss : 1.5819, Accuracy : 0.8793


In [22]:
# Pandas로 정리하여 보기 위해 List로 정리하기
model1Result = [train_accuracy, val_accuracy, test_accuracy]
model1Result

[0.9164166666666667, 0.88175, 0.8793]

---
### Model2
입력층 -> 은닉층(활성화함수) -> Dropout층 -> 출력층 

In [32]:
# 훈련 평가
train_loss, train_accuracy = evaluate(model2, train_loader, criterion2, device)
print(f"Loss : {train_loss:.4f}, Accuracy : {train_accuracy:.4f}")

Loss : 1.5759, Accuracy : 0.8849


In [33]:
# 검증 평가
val_loss, val_accuracy = evaluate(model2, val_loader, criterion2, device)
print(f"Loss : {val_loss:.4f}, Accuracy : {val_accuracy:.4f}")

Loss : 1.5938, Accuracy : 0.8665


In [34]:
# 일반화 평가
test_dataset = TensorDataset(test_scaled, test_target)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

test_loss, test_accuracy = evaluate(model2, test_loader, criterion2, device)
print(f"Loss : {test_loss:.4f}, Accuracy : {test_accuracy:.4f}")

Loss : 1.5984, Accuracy : 0.8619


In [35]:
# Pandas로 정리하여 보기 위해 List로 정리하기
model2Result = [train_accuracy, val_accuracy, test_accuracy]
model2Result

[0.8849375, 0.8665, 0.8619]

---
### 모델 3
입력층 -> 은닉층(활성화 함수) -> Dropout(0.2) ->  은닉층(활성화 함수) -> 출력층 

In [41]:
# 훈련 평가
train_loss, train_accuracy = evaluate(model3, train_loader, criterion3, device)
print(f"Loss : {train_loss:.4f}, Accuracy : {train_accuracy:.4f}")

Loss : 1.5660, Accuracy : 0.8951


In [42]:
# 검증 평가
val_loss, val_accuracy = evaluate(model3, val_loader, criterion3, device)
print(f"Loss : {val_loss:.4f}, Accuracy : {val_accuracy:.4f}")

Loss : 1.5842, Accuracy : 0.8767


In [43]:
# 일반화 평가
test_dataset = TensorDataset(test_scaled, test_target)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

test_loss, test_accuracy = evaluate(model3, test_loader, criterion3, device)
print(f"Loss : {test_loss:.4f}, Accuracy : {test_accuracy:.4f}")

Loss : 1.5902, Accuracy : 0.8706


In [44]:
# Pandas로 정리하여 보기 위해 List로 정리하기
model3Result = [train_accuracy, val_accuracy, test_accuracy]
model3Result

[0.8950625, 0.8766666666666667, 0.8706]

In [45]:
# Model 4 List
model4Result = [0.89, 0.87, 0.86]
model4Result 

[0.89, 0.87, 0.86]

---

In [46]:
modelResult = [model1Result, model2Result, model3Result, model4Result]
modelResult

[[0.9164166666666667, 0.88175, 0.8793],
 [0.8849375, 0.8665, 0.8619],
 [0.8950625, 0.8766666666666667, 0.8706],
 [0.89, 0.87, 0.86]]

In [47]:
import pandas as pd

pd.DataFrame(
    modelResult,
    columns=['Train', 'Valid', 'Test'],
    index=['Model1', 'Model2', 'Model3', 'Model4']
)

,Train,Valid,Test
Model1,0.916417,0.881750,0.8793
Model2,0.884938,0.866500,0.8619
Model3,0.895062,0.876667,0.8706
Model4,0.890000,0.870000,0.8600
